In [1]:
#imports
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as ec
from collections import deque
from selenium.webdriver.common.keys import Keys
import time

In [2]:
#Chromedriver
browser = webdriver.Chrome()
browser.get('https://web.telegram.org/#/im')

In [13]:
#Lets the user select groups to track
def selectGroups(groupChats):
    index = 1
    for chat in groupChats:
        title= chat.find_elements_by_css_selector("div.im_dialog_peer")[0].text
        print(str(index) + '. ' + title)
        index+=1
    
    #once chats are printed, ask user for csv of indexes of groupchats they want to track
    listOfIndexOfChats = []
    while True:
        try:
            groupNumber = int(input("Please input the number of"+
                                    " the groupchat you want to track (any non-number to proceed): "))
            listOfIndexOfChats.append(groupNumber)
        except ValueError:
            break
            
    #now that we have the group indeces, use global variable to change indeces to chatNames
    userChats = []
    for index in listOfIndexOfChats:
        userChats.append(groupChats[index-1])
    print('You have selected the following groups: ')
    #return userChats
    return userChats
    
#________________________________________________________________________#
def selectHomeGroup(groupChats):
    index = int(input("Please input the number of the Home Group: "))
    homeGroup = groupChats[index-1].find_elements_by_css_selector("div.im_dialog_peer")[0].text
    print('Selected '+ homeGroup+' as homegroup.')
    return homeGroup

#________________________________________________________________________#
#Creates dictionairy of last messages in each group
def initializeDict(groupChats):
    lastMessageDict = {}
    for chat in groupChats:
        chat.click()
        selectedGroupTitle = chat.find_elements_by_css_selector("div.im_dialog_peer")[0].text
        while True:
            try:
                messages = browser.find_elements_by_class_name("im_message_text")
                textMessages = []
                #change array of WebElements to array of strings. easier to work with, avoid errors
                for webElement in messages:
                    textMessages.append(webElement.text)
                #for some reason there were empty strings being grabbed by selenium. removing them here    
                textMessages = list(filter(None, textMessages))
                key_gc = selectedGroupTitle
                lastMessageDict[key_gc] = textMessages[-1]
                print('Loaded ' + selectedGroupTitle + ': '+ lastMessageDict[key_gc])
                break
            except:
                print(selectedGroupTitle + 'did not load yet, retrying.')
    print(lastMessageDict)
    print('Recorded most recent messages in every group. New messages will be pushed to homegroup.')
    return lastMessageDict
#________________________________________________________________________#
#groupQueueBuilder
def groupQueueBuilder(trackedChats):
    for groupChat in trackedChats:
        oneBadgeArray = groupChat.find_elements_by_css_selector("span.im_dialog_badge.badge")
#         TO DO : make it not take messages from HOME GROUP
        if(oneBadgeArray[0].text!=""):
            #get index and use this index in titlesArray
            oneTitleArray = groupChat.find_elements_by_css_selector("div.im_dialog_peer")
            if(oneTitleArray[0].text not in groupQueue):
                groupQueue.append(oneTitleArray[0].text)
    print(groupQueue)
    return groupQueue
#________________________________________________________________________#
#groupFinder - Takes in name of group finds it selects it, returns title of slected chat
def groupFinder(name, trackedChats):
    print('Finding '+ name)
    for chat in trackedChats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == name):
            chat.click()
            print('Clicked on ' + name)
            return(titleArray[0].text)
    print('groupFinder Could not find group: '+ name)
    return 'noGroupError'
    
#________________________________________________________________________#
#Create push method to click on 'home chat' and paste and send it there.
# checks via dictionary which messages to push

def createMessageStack(selectedGroupTitle, lastMessageDict):
    linkedGroupChatContainer = browser.find_element_by_class_name('im_history_loaded')
#     print(linkedGroupChatContainer)
    groupChatContainerElement = linkedGroupChatContainer.find_elements_by_class_name("im_history_messages_peer")
#     print(groupChatContainerElement)
    for element in groupChatContainerElement:
        if(element.get_attribute("class") == 'im_history_messages_peer'): 
            messages = element.find_elements_by_class_name("im_message_text")
#             print(messages)
    textMessages = []
    print('Loading '+str(len(messages))+' new messages')
    #change array of WebElements to array of strings. easier to work with, avoid errors
    for webElement in messages:
    #     if(len(webElement.text) > 1):
#         print(webElement.text)
        textMessages.append(webElement.text)
    #for some reason there were empty strings being grabbed by selenium. removing them here    
    textMessages = list(filter(None, textMessages))
    textMessages.reverse()
    ##
    messageStack = []
    for message in textMessages:
        if(message != lastMessageDict[selectedGroupTitle]):
            messageStack.append(message)
        else:
#             SHOULD RUN WHEN MESSAGES ARE PUSHED
            break
    print((str)(len(messageStack)) + ' messages Loaded')
    print(messageStack)
    return messageStack
    ##
    print(textMessages)

#DEPRECATED METHOD
# def createMessageStack(selectedGroupTitle, lastMessageDict):
# #     groupChatContainerElement = browser.find_element_by_class_name("im_history_messages_peer")
#     messageStack = []
#     messages = browser.find_elements_by_class_name("im_message_text")
#     textMessages = []
#     print('Loading '+str(len(messages))+' new messages')
#     #change array of WebElements to array of strings. easier to work with, avoid errors
#     for webElement in messages:
#         if(len(webElement.text) > 1):
#             print(webElement.text)
#         textMessages.append(webElement.text)
#     #for some reason there were empty strings being grabbed by selenium. removing them here    
#     textMessages = list(filter(None, textMessages))
#     #reversing list to make FIFO
#     textMessages.reverse()
#     for message in textMessages:
#         if(message != lastMessageDict[selectedGroupTitle]):
#             messageStack.append(message)
#         else:
# #             SHOULD RUN WHEN MESSAGES ARE PUSHED
#             break
#     print((str)(len(messageStack)) + ' messages Loaded')
#     print(messageStack)
#     return messageStack
 #________________________________________________________________________#
# Broadcast Method
def broadCast(messageStack, selectedGroupTitle, groupChats, homeGroup, dictionairy):
    #select home chat
    print('Sending...')
    print(messageStack)
    messageStack.reverse()
    for chat in groupChats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == homeGroup):
            chat.click()
            break
    
    #once homechat is selected, send messages one by one.
    textBox = browser.find_elements_by_class_name('composer_rich_textarea')[0]
    for message in messageStack:        
        textBox.send_keys(selectedGroupTitle + ": " + message)
        textBox.send_keys(Keys.RETURN)
    if(len(messageStack) != 0):
        dictionairy[selectedGroupTitle] = messageStack[-1]
    #clean up after sending messages.
    messageStack.clear()
    print('Messages sent!')
    #print(messageStack)
    return 0
#________________________________________________________________________#
def fillGroupQueue(trackedChats):
    groupQueue = deque([])
    for webElement in trackedChats:
        titleArray = webElement.find_elements_by_css_selector("div.im_dialog_peer")
        groupQueue.append(titleArray[0].text)
    print('Time to check every group for missed messages')
    print(groupQueue)
    return groupQueue
        
#________________________________________________________________________#
#User prompt to run BotLoop
global READY
READY = False

while True:
    print('______________________________________________________________')
    print('READ DIRECTIONS CAREFULLY.')
    try:
        userPrompt = input("Please type 'ready' after signing in to continue. ")            
        
    except ValueError:
        print("You gotta type in 'ready' exactly like that, bud...")
        continue

    else:
        if(userPrompt == 'ready'):
            READY = True
            break
        else:
            continue

#________________________________________________________________________#
#BotLoop
#grabs all groupchats

# while True:
#     print('inside botLoop')
# try:
if (READY):
    #Create Runtime Variables inside here.
    groupChats = browser.find_elements_by_class_name("im_dialog_wrap") 
    groupQueue = deque([])
    trackedChats = selectGroups(groupChats)
    homeGroup = selectHomeGroup(groupChats)
    lastMessageDict = initializeDict(trackedChats)
    timeout = 3600
    timeout_start = time.time()
    cleanupTimeout = time.time()
    # month timeout:
    # while time.time() < 1517278786 + 259200:
    while time.time() < timeout_start + timeout:
        if(len(list(groupQueue)) == 0):
            time.sleep(1)
            if(time.time() > cleanupTimeout + 10):
                groupQueue = fillGroupQueue(trackedChats)
                cleanupTimeout = time.time()
            else:
                groupQueue = groupQueueBuilder(trackedChats)
                if(len(list(groupQueue)) != 0):
                    cleanupTimeout = time.time()
                
        else:
            selectedGroupTitle = groupFinder(groupQueue.popleft(), trackedChats)
            messageStack = createMessageStack(selectedGroupTitle, lastMessageDict)
            broadCast(messageStack, selectedGroupTitle, groupChats,homeGroup, lastMessageDict)
            cleanupTimeout = time.time()

    print('Finished BotLoop')
    
    
        
        
            #get first in queue and cache / print all messages from it.
            #For our purpose, batch work scheduler would be better. **prioritizes active groupchats**
    # except:
        # pass


        

______________________________________________________________
READ DIRECTIONS CAREFULLY.
Please type 'ready' after signing in to continue. ready
1. WhaleClub (Bitcoin ONLY) Traders
2. The Coin Farm
3. FGCU CRYPTO CRIPS
4. PAID GROUP
5. Telegram
6. DeCenter – Blockchain and Bitcoin
7. Cointelegraph
8. Jason Cas
9. :palm_tree:Gulf Coast Signals
10. Tips
11. Danny Lemur
12. Austin
13. Whale Group
14. Telecast
15. Program
16. Rafalllll
Please input the number of the groupchat you want to track (any non-number to proceed): 1
Please input the number of the groupchat you want to track (any non-number to proceed): 2
Please input the number of the groupchat you want to track (any non-number to proceed): 3
Please input the number of the groupchat you want to track (any non-number to proceed): 6
Please input the number of the groupchat you want to track (any non-number to proceed): 8
Please input the number of the groupchat you want to track (any non-number to proceed): s
You have selected the f

Messages sent!
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 48 new messages
2 messages Loaded
['Is BTC ever stable lololol', "But it's back up now"]
Sending...
['Is BTC ever stable lololol', "But it's back up now"]
Messages sent!
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 48 new messages
2 messages Loaded
['^', 'hi yolo']
Sending...
['^', 'hi yolo']
Messages sent!
deque([])
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 48 new messages
1 messages Loaded
['Sup']
Sending...
['Sup']
Messages sent!
deque([])
deque([])
deque([])
deque([])
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 48 new messages
1 messages Loaded
['Bitcoin when 2k']
Sending...
['Bitcoin when 2k']
Messages sent!
deque([])
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 47 new messages
1 messages Loaded
['I need bitcoin']
Sending...
['I need bitcoin']
Me

Loading 49 new messages
2 messages Loaded
["as u have seen many people said  it's going to 1k 2k", 'Binance listing wrecked you']
Sending...
["as u have seen many people said  it's going to 1k 2k", 'Binance listing wrecked you']
Messages sent!
deque(['WhaleClub (Bitcoin ONLY) Traders'])
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 44 new messages
1 messages Loaded
['They don’t allow using their credit card to buy crypto']
Sending...
['They don’t allow using their credit card to buy crypto']
Messages sent!
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
Time to check every group for missed messages
deque(['WhaleClub (Bitcoin ONLY) Traders', 'The Coin Farm', 'FGCU CRYPTO CRIPS', 'DeCenter – Blockchain and Bitcoin', 'Jason Cas'])
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 44 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
Finding The Co

Messages sent!
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 49 new messages
1 messages Loaded
['Chinese New Year holidays kicking off here so still a few weeks before that finishes up']
Sending...
['Chinese New Year holidays kicking off here so still a few weeks before that finishes up']
Messages sent!
deque([])
deque([])
deque([])
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 50 new messages
2 messages Loaded
['india', 'I will do as well.. fuckin missed 2 train in a week']
Sending...
['india', 'I will do as well.. fuckin missed 2 train in a week']
Messages sent!
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 50 new messages
2 messages Loaded
['i can increase my btc', 'but atleast']
Sending...
['i can increase my btc', 'but atleast']
Messages sent!
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 50 new messages
1 messages Loaded
['Have u looked into your

Messages sent!
Finding Jason Cas
Clicked on Jason Cas
Loading 21 new messages
0 messages Loaded
[]
Sending...
[]
Messages sent!
deque([])
deque([])
deque([])
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 49 new messages
5 messages Loaded
['Now I just troll', 'Ended up hitting it big', 'I got into real estate when I was young', '2 week crypto break sounds nice', 'ya']
Sending...
['Now I just troll', 'Ended up hitting it big', 'I got into real estate when I was young', '2 week crypto break sounds nice', 'ya']
Messages sent!
deque([])
deque(['WhaleClub (Bitcoin ONLY) Traders'])
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 43 new messages
1 messages Loaded
['Not allowed to post links']
Sending...
['Not allowed to post links']
Messages sent!
deque([])
deque([])
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 49 new messages
1 messages Loaded
['yolo.. today will be with 9k ?']
Se

Messages sent!
deque(['The Coin Farm'])
Finding The Coin Farm
Clicked on The Coin Farm
Loading 49 new messages
6 messages Loaded
['we are all here to make money and help each other', ':joy:', 'Admin warned you earlier to stop this yolo', 'Behrad is permabear', 'instead of keeping iin hardware wallet', 'I would have made more btc']
Sending...
['we are all here to make money and help each other', ':joy:', 'Admin warned you earlier to stop this yolo', 'Behrad is permabear', 'instead of keeping iin hardware wallet', 'I would have made more btc']
Messages sent!
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
deque([])
Time to check every group for missed messages
deque(['WhaleClub (Bitcoin ONLY) Traders', 'The Coin Farm', 'FGCU CRYPTO CRIPS', 'DeCenter – Blockchain and Bitcoin', 'Jason Cas'])
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 43 new messages
7 messages Loaded
['Cheers', 'looks like its going to test 7

WebDriverException: Message: unknown error: ChromeDriver only supports characters in the BMP
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.13.2 x86_64)


In [8]:
def createMessageStack(selectedGroupTitle, lastMessageDict):
    linkedGroupChatContainer = browser.find_element_by_class_name('im_history_loaded')
#     print(linkedGroupChatContainer)
    groupChatContainerElement = linkedGroupChatContainer.find_elements_by_class_name("im_history_messages_peer")
#     print(groupChatContainerElement)
    for element in groupChatContainerElement:
        if(element.get_attribute("class") == 'im_history_messages_peer'): 
            messages = element.find_elements_by_class_name("im_message_text")
#             print(messages)
    textMessages = []
    print('Loading '+str(len(messages))+' new messages')
    #change array of WebElements to array of strings. easier to work with, avoid errors
    for webElement in messages:
    #     if(len(webElement.text) > 1):
#         print(webElement.text)
        textMessages.append(webElement.text)
    #for some reason there were empty strings being grabbed by selenium. removing them here    
    textMessages = list(filter(None, textMessages))
    textMessages.reverse()
    ##
    messageStack = []
    for message in textMessages:
        if(message != lastMessageDict[selectedGroupTitle]):
            messageStack.append(message)
        else:
#             SHOULD RUN WHEN MESSAGES ARE PUSHED
            break
    print((str)(len(messageStack)) + ' messages Loaded')
    print(messageStack)
    return messageStack
    ##
    print(textMessages)

Loading 44 new messages
42 messages Loaded
["I think I've learn more from Tai Lopez", 'I thought this was a serious telegram :persevere:', 'Wait for 2000$', '7000$ is coming', 'If its not now then why there is soo many bitcoin atm that been distributed around the banks!!!!', 'Get an avatar please', 'lol', 'A lot of retards too', 'however, priob just scammers who have shorts on atm', 'Nevery seen so many scared ppl in  a single chat room before', 'I think so too but not now. So I short it', 'Thats something i cant predict but by the end of 2018 or begining of 2019 yes', 'you must be make some profit this very moment', 'Good point but do you think the time is now?', 'so I guess when there´s a crash, people think it will be back in a couple of days.... but those days might become a couple of years :expressionless:', 'Welcome Mikhail, please read the rules and introduction on https://whaleclub.io/whaleclub-main-rooms/', 'They need a gold standart. In the digitalized money. Just like real g